# Data Exploration and Cleaning

In this notebook and as this part of the process, we will pursue a high level understanidng of the data and do some cleaning to make sure we don't have any dirty data at the beginning of our analysis. This inclues looking at columns and data types, ensuring these data types are correct, handling missing or null values in the data, and engineering the initial set of features for our analysis and eventual model building. 

## Import Libraries and Data

In [87]:
import pandas as pd 
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer

In [74]:
import os 
print(os.getcwd())
os.chdir('C:/Users/Joseph/source/repos/Kaggle-SpaceShip-Titanic-Competition-1/')
print(os.getcwd())

C:\Users\Joseph\source\repos\Kaggle-SpaceShip-Titanic-Competition-1
C:\Users\Joseph\source\repos\Kaggle-SpaceShip-Titanic-Competition-1


## View the Data

In [75]:
sample_submission = pd.read_csv('sample_submission.csv.crdownload')
sample_submission

,PassengerId,Transported
0,0013_01,False
1,0018_01,False
2,0019_01,False
3,0021_01,False
4,0023_01,False
...,...,...
3166,6935_01,False
3167,6936_01,False
3168,6936_02,False
3169,6936_03,False


In [76]:
test_df = pd.read_csv('test.csv')
train_df = pd.read_csv('train.csv')

train_df.head(10)

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True
5,0005_01,Earth,False,F/0/P,PSO J318.5-22,44.0,False,0.0,483.0,0.0,291.0,0.0,Sandie Hinetthews,True
6,0006_01,Earth,False,F/2/S,TRAPPIST-1e,26.0,False,42.0,1539.0,3.0,0.0,0.0,Billex Jacostaffey,True
7,0006_02,Earth,True,G/0/S,TRAPPIST-1e,28.0,False,0.0,0.0,0.0,0.0,NaN,Candra Jacostaffey,True
8,0007_01,Earth,False,F/3/S,TRAPPIST-1e,35.0,False,0.0,785.0,17.0,216.0,0.0,Andona Beston,True
9,0008_01,Europa,True,B/1/P,55 Cancri e,14.0,False,0.0,0.0,0.0,0.0,0.0,Erraiam Flatic,True


Looking at the top ten rows from the dataframe above, we can see that our output variable is Transported while our other variables are predictors. 

## Explore the data at a high level

As the first part of our initial exploration, we can look at the data types as well as the general structure of the dataset- specifically looking at the train data with the understanding that we can adjust the training data once we have our null values handled and our initial features identified. 

In [36]:
train_df.dtypes

PassengerId      object
HomePlanet       object
CryoSleep        object
Cabin            object
Destination      object
Age             float64
VIP              object
RoomService     float64
FoodCourt       float64
ShoppingMall    float64
Spa             float64
VRDeck          float64
Name             object
Transported        bool
dtype: object

We see that while Cryosleep and VIP are defined as object dtypes, the are boolean data types when looking at the actual data, so we can adjust these two right away. 

And while we are working through the adjustments in train_df, we will do the same thing with the test_df. 

In [77]:
train_df["CryoSleep"] = train_df["CryoSleep"].astype(bool)
test_df["CryoSleep"] = test_df["CryoSleep"].astype(bool)
train_df["VIP"] = train_df["VIP"].astype(bool)
test_df["VIP"] = test_df["VIP"].astype(bool)

train_df.dtypes

PassengerId      object
HomePlanet       object
CryoSleep          bool
Cabin            object
Destination      object
Age             float64
VIP                bool
RoomService     float64
FoodCourt       float64
ShoppingMall    float64
Spa             float64
VRDeck          float64
Name             object
Transported        bool
dtype: object

We also know that our column Cabin contains three helpful pieces of information that would be helpful to split into three separate columns so we can use the information provided. These three pieces of information are deck, number, and side, which is either port or starboard. We will split this column into three new columns called deck, num, and side. 

In [78]:
train_df[["Deck","Number","Side"]] = train_df["Cabin"].str.split('/',expand=True)
train_df.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,Deck,Number,Side
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False,B,0,P
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True,F,0,S
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False,A,0,S
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False,A,0,S
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True,F,1,S


In [79]:
test_df[["Deck","Number","Side"]] = test_df["Cabin"].str.split('/',expand=True)
test_df.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Deck,Number,Side
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning,G,3,S
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers,F,4,S
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus,C,0,S
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter,C,1,S
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez,F,5,S


Two other features we could extract at this point would be the group and the passenger's number within the group. The two features will come from the passenger ID which is separated by the '_', with the group number being on the left side and the number within the group being on the right. 

In [80]:
test_df[["Group","Num_in_Group"]] = test_df["PassengerId"].str.split('_',expand=True)
test_df.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Deck,Number,Side,Group,Num_in_Group
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning,G,3,S,0013,01
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers,F,4,S,0018,01
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus,C,0,S,0019,01
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter,C,1,S,0021,01
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez,F,5,S,0023,01


In [42]:
train_df[["Group","Num_in_Group"]] = train_df["PassengerId"].str.split('_',expand=True)
train_df.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,deck,number,side,Group,Num_in_Group
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False,B,0,P,0001,01
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True,F,0,S,0002,01
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False,A,0,S,0003,01
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False,A,0,S,0003,02
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True,F,1,S,0004,01


Now that we have split our combined features into new features, we can look over our data types once more and make any adjustments. 

In [43]:
train_df.dtypes

PassengerId      object
HomePlanet       object
CryoSleep          bool
Cabin            object
Destination      object
Age             float64
VIP                bool
RoomService     float64
FoodCourt       float64
ShoppingMall    float64
Spa             float64
VRDeck          float64
Name             object
Transported        bool
deck             object
number           object
side             object
Group            object
Num_in_Group     object
dtype: object

Our data types look good. Although Number, Group, and Num_in_Group look like numbers these really are strings that indicate some kind of category. 

## Look for Null variables in the Training data

We will look at our training data to identify the number of null values by column to determine whether we should remove or impute. One element to consider is the number of null variables that exist and decide at what point and what kind of impact imputation would have on our accuracy. If too many null variables receive imputed values, that could impact our accuracy. 

The first thing we should do is determine the relative sizes of the training and testing datasets. 

In [81]:
print(f'Training dataset size: {train_df.shape}')

Training dataset size: (8693, 17)


In [82]:
print(f'Testing dataset size: {test_df.shape}')

Testing dataset size: (4277, 18)


The explanation for the missing column in the testing dataset is the output variable does not exist. That is the column we are to predict for the testing dataset. 

We see that our training dataset has 19 columns and 8,693 individual records. We will keep these numbers in mind as we look through the null values in our training dataset. 

In [83]:
for col in train_df.columns:
    print(f'{col}: {train_df[col].isna().sum()}')


PassengerId: 0
HomePlanet: 201
CryoSleep: 0
Cabin: 199
Destination: 182
Age: 179
VIP: 0
RoomService: 181
FoodCourt: 183
ShoppingMall: 208
Spa: 183
VRDeck: 188
Name: 200
Transported: 0
Deck: 199
Number: 199
Side: 199


Taking into consideration the fact that deck, number, and side will all have the same number null values as Cabin, we see that we do have a decent number of null values. One thing to examine would be crossover records: 
- Do any of the records with null values in one column also contain null values within another column? 

We can examine this by counting the number of rows with null values and then comparing that answer to the total number of null values in the dataset- minus the columns for deck, number, and side as these will receive whatever imputation Cabin receives. 

In [84]:
total_nulls = 0
for i in range(14):
    total_nulls += train_df.iloc[:,i].isna().sum()
print(f'Total null values in training dataset: {total_nulls}')

Total null values in training dataset: 1904


In [85]:
total_null_rows = train_df.isna().any(axis=1).sum()
print(f'Total rows with null values in training dataset: {total_null_rows}')

Total rows with null values in training dataset: 1750


In [86]:
print(f'{total_nulls - total_null_rows} rows have more than one null value')

154 rows have more than one null value


So all but 154 rows have only one null value, which indicates a low level of crossover of null values from one column to the next on the same row. 

We have less than 2.5% of any given column as null values, so we can probably impute values to the missing data. We will approach the null values in the following manner:
- Numerical data: for the numerical data, we will impute the median value as this will help avoid any skewing from larger numbers.  
- Categorical data: for the categorical data, we will simply assign the value "Unknown" to the missing values and create a new category. 
- Flag Columns: lastly, we will create flag columns for the columns that have missing values using a binary assignment as this can help imputation-aware models. 

With these done, we should no longer have any null values in the data and we can then decide if we have any other further features we would like to engineer. 

In [89]:
numerical_cols = ["Age","RoomService","FoodCourt","ShoppingMall","Spa","VRDeck"]

imputer_num = SimpleImputer(strategy='median')
train_df[numerical_cols] = imputer_num.fit_transform(train_df[numerical_cols])
test_df[numerical_cols] = imputer_num.fit_transform(test_df[numerical_cols])

In [92]:
categorical_cols = ["HomePlanet","Cabin","Destination","deck","number","side","Group","Num_in_Group"]

imputer_cat = SimpleImputer(strategy='most_frequent')
train_df[categorical_cols] = imputer_cat.fit_transform(train_df[categorical_cols])
test_df[categorical_cols] = imputer_cat.fit_transform(test_df[categorical_cols])

KeyError: "['deck', 'number', 'side', 'Group', 'Num_in_Group'] not in index"

In [ ]:
for col in numerical_cols + categorical_cols:
    train_df[col + '_missing'] = train_df[col].isnull().astype(int)

for col in train_df.columns:
    print(f'{col}: {train_df[col].isna().sum()}')